In [ ]:
!pip install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nest_asyncio
nest_asyncio.apply()
import asyncio 
from multiprocessing import Pool, cpu_count
import concurrent.futures
import sys
import time
from enum import Enum
import aiohttp
import requests

In [ ]:
def get_ids_cervejarias():
    r = requests.get('https://api.openbrewerydb.org/breweries')
    if r.status_code == 200:
        return [r['id'] for r in r.json()]

In [ ]:
def get_cervejaria(id_c):
    r = requests.get(f'https://api.openbrewerydb.org/breweries/{id_c}')
    if r.status_code == 200:
        print(r.json())

In [ ]:
async def get_cervejaria_async(id_c, session):
    async with session.get(f'https://api.openbrewerydb.org/breweries/{id_c}') as response:
        print(await response.json())


In [ ]:
async def processar_async(ids):
    coros = []
    async with aiohttp.ClientSession() as session:
        for id_c in ids: 
            coros.append(get_cervejaria_async(id_c, session))

        await asyncio.gather(*coros)

In [ ]:
async def get_cervejaria_async_manual(id_c):
    loop = asyncio.get_running_loop()
    return await loop.run_in_executor(None, get_cervejaria, id_c)

In [ ]:
class TipoExecucao(Enum):
    UM_PROCESSO = 1
    VARIAS_THREADS = 2
    ASYNCIO_COM_LIB_HTTP = 3

In [ ]:
if __name__ == '__main__':
    started = time.time()

    tipo_execucao = TipoExecucao.ASYNCIO_COM_LIB_HTTP
    cores = cpu_count()

    ids_cervejarias = get_ids_cervejarias()

    if TipoExecucao.UM_PROCESSO == tipo_execucao:
        # Usando somente um processo
        for id_cervejaria in ids_cervejarias:
            get_cervejaria(id_cervejaria)

    elif TipoExecucao.VARIAS_THREADS == tipo_execucao:
        # Usando varias threads
        with concurrent.futures.ThreadPoolExecutor(cores) as thp:
            thp.map(get_cervejaria, ids_cervejarias)

    elif TipoExecucao.ASYNCIO_COM_LIB_HTTP == tipo_execucao:
        # Usando asyncio
        event_loop = asyncio.get_event_loop()
        tasks = []

        event_loop.run_until_complete(processar_async(ids_cervejarias))

    elif TipoExecucao.ASYNCIO_MANUAL == tipo_execucao:
        # Usando asyncio manual
        event_loop = asyncio.get_event_loop()
        tasks = []

        for id_cervejaria in ids_cervejarias:
            tasks.append(get_cervejaria_async_manual(id_cervejaria))

        event_loop.run_until_complete(asyncio.wait(tasks))

    elapsed = time.time()
    print('Time taken :', elapsed - started)
        

{'id': '10-barrel-brewing-co-bend-2', 'name': '10 Barrel Brewing Co', 'brewery_type': 'large', 'street': '1135 NW Galveston Ave Ste B', 'address_2': None, 'address_3': None, 'city': 'Bend', 'state': 'Oregon', 'county_province': None, 'postal_code': '97703-2465', 'country': 'United States', 'longitude': '-121.32880209261799', 'latitude': '44.057564901366796', 'phone': '5415851007', 'website_url': None, 'updated_at': '2022-11-11T05:07:58.723Z', 'created_at': '2022-11-11T05:07:58.723Z'}
{'id': '12-west-brewing-company-production-facility-mesa', 'name': '12 West Brewing Company - Production Facility', 'brewery_type': 'micro', 'street': None, 'address_2': None, 'address_3': None, 'city': 'Mesa', 'state': 'Arizona', 'county_province': None, 'postal_code': '85207', 'country': 'United States', 'longitude': '-111.5860662', 'latitude': '33.436188', 'phone': None, 'website_url': None, 'updated_at': '2022-11-11T05:07:58.723Z', 'created_at': '2022-11-11T05:07:58.723Z'}
{'id': '101-brewery-quilcene'